# Tutorial 9: Weighted Passages
## Enriched Categories and Probabilistic Structure

---

### Beyond Yes and No

*To the research assistant:*

*In Year 901, Tessery Vold made a crucial refinement to her passage diagrams. The original framework asked only: "Does a passage exist from A to B?" But observation revealed a richer structure.*

*Not all passages are equal. Stakdurs traverse from territory to reed marsh daily—this is a well-worn path. But the passage from deep Dens to Capital outskirts is rare, observed perhaps once in a decade. To capture this, Vold proposed "weighted passages"—morphisms decorated with degrees.*

*Instead of Hom(A, B) being a set (either empty or containing morphisms), Vold proposed that Hom(A, B) could be a number: a probability, a frequency, a cost. This transforms the passage category into what modern mathematicians call an "enriched category."*

*Marden Krell grudgingly admitted this was an improvement: "At least now we can distinguish common events from rare ones. But you still have not explained why these passages exist at all."*

*Your task: Analyze weighted passage data to understand enriched categorical structure. Determine how weights affect composition, identity, and the overall behavior of the category.*

—*Archive Review Committee, Year 934*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Understand **enriched categories**: morphisms with values, not just existence
2. Model **probabilistic morphisms** and their composition
3. Connect enrichment to **weighted graphs** and **Markov chains**
4. See how enriched categories relate to **attention mechanisms** in neural networks
5. Apply weighted reasoning to passage diagram data

By the end, you will understand:
- Why "how much" matters as much as "whether"
- The categorical foundation of probabilistic models
- How attention mechanisms can be viewed as enriched functors

---

## What Is an Enriched Category?

In an ordinary category:
- Hom(A, B) is a **set** of morphisms

In an enriched category:
- Hom(A, B) is an **object in some other category V**

Common enrichments:
- **V = [0, ∞]**: Hom(A, B) = distance/cost from A to B (metric spaces)
- **V = [0, 1]**: Hom(A, B) = probability of passing from A to B
- **V = ℝ**: Hom(A, B) = similarity or weight between A and B
- **V = Bool**: Hom(A, B) = true/false (this recovers pre-orders)

### Composition in Enriched Categories

Composition must respect the enrichment:
- For distances: Hom(A, C) ≤ Hom(A, B) + Hom(B, C) (triangle inequality)
- For probabilities: Hom(A, C) ≥ Hom(A, B) × Hom(B, C) (chain rule)

### Vold's Interpretation

> *"A passage is not merely a yes or no. The stakdur's daily hunt is a certainty; the grimslew's rare emergence is a whisper. To capture the world's structure, we must weight our passages."*

---

## Part 1: Loading and Preparing Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from collections import defaultdict

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')

print("Libraries loaded. Ready to study weighted passages.")

In [ ]:
# Load passage diagram data
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

passages = pd.read_csv(BASE_URL + "passage_diagrams.csv")

print(f"Passages loaded: {len(passages)} morphisms")
print(f"\nColumns: {list(passages.columns)}")

In [ ]:
# The 'frequency' column provides weight information
print("Frequency values in data:")
print(passages['frequency'].value_counts())

The frequency values describe how often passages occur. Let's convert these to numerical weights for our enriched category.

---

## Part 2: Converting to Numerical Weights

In [ ]:
# Define a mapping from frequency descriptions to probabilities
frequency_to_probability = {
    'constant': 1.0,      # Always happens
    'daily': 0.95,        # Very frequent
    'weekly': 0.8,        # Frequent
    'dawn': 0.85,         # Regular timing
    'dusk': 0.85,
    'circadian': 0.9,     # Daily cycle
    'tidal': 0.75,        # Periodic
    'overnight': 0.7,
    'monthly': 0.5,       # Moderate
    'seasonal': 0.4,      # Less frequent
    'annual': 0.3,
    'per_submission': 0.6,
    'per_acceptance': 0.5,
    'achievement': 0.3,   # Milestone events
    'production': 0.7,
    'developmental': 0.5,
    'maturity': 0.3,
    'inductive': 0.6,
    'deductive': 0.5,
    'confirmatory': 0.4,
    'interactive': 0.7,
    'variable': 0.3,      # Unpredictable
    'episodic': 0.2,
    'cascading': 0.15,
    'rare': 0.1,          # Very rare
    'irreversible': 0.05, # One-time
    'mortality': 0.02,
    'terminal': 0.01,
    'temporal': 0.3,
    'never': 0.0,         # Impossible
    'unknown': 0.5        # Default to moderate
}

# Add probability column
passages['weight'] = passages['frequency'].map(
    lambda x: frequency_to_probability.get(x, 0.5)
)

print("Weight distribution:")
print(passages['weight'].describe())

In [ ]:
# Visualize weight distribution
fig, ax = plt.subplots(figsize=(10, 5))
passages['weight'].hist(bins=20, ax=ax, color='steelblue', alpha=0.7)
ax.set_xlabel('Weight (Probability)')
ax.set_ylabel('Count')
ax.set_title('Distribution of Passage Weights')
plt.tight_layout()
plt.show()

Now we have a numerical weight for each passage. This is our enriched morphism: not just "does a passage exist?" but "how probable/frequent is it?"

---

## Part 3: Building the Enriched Category

In an enriched category, Hom(A, B) is a value (here, a probability), not a set.

In [ ]:
# Build the enriched Hom structure
# For simplicity, if multiple morphisms exist between A and B, take the max probability

all_objects = sorted(set(passages['source_object']) | set(passages['target_object']))
n = len(all_objects)
obj_to_idx = {obj: i for i, obj in enumerate(all_objects)}

# Initialize Hom matrix with 0 (no passage)
Hom = np.zeros((n, n))

# Fill in from data
for _, row in passages.iterrows():
    i = obj_to_idx[row['source_object']]
    j = obj_to_idx[row['target_object']]
    Hom[i, j] = max(Hom[i, j], row['weight'])

# Set identity morphisms to 1 (certainty)
np.fill_diagonal(Hom, 1.0)

print(f"Enriched Hom matrix: {Hom.shape}")
print(f"Non-zero entries: {np.count_nonzero(Hom)} (including {n} identities)")

In [ ]:
# Visualize a portion of the Hom matrix
fig, ax = plt.subplots(figsize=(12, 10))

# Show first 15 objects
k = 15
sns.heatmap(Hom[:k, :k], annot=True, fmt='.2f', cmap='Blues',
            xticklabels=all_objects[:k], yticklabels=all_objects[:k],
            ax=ax)
ax.set_title('Enriched Hom Matrix (Probability of Passage)\nFirst 15 Objects')
ax.set_xlabel('Target')
ax.set_ylabel('Source')
plt.tight_layout()
plt.show()

Each entry Hom[i, j] is the probability/weight of passing from object i to object j. The diagonal is all 1s (identity morphisms have weight 1).

---

## Part 4: Composition in Enriched Categories

In a probability-enriched category, composition is multiplication:

If P(A → B) = p and P(B → C) = q, then P(A → C via B) = p × q

But there may be multiple paths, so we often take max:

Hom(A, C) = max over all B of [Hom(A, B) × Hom(B, C)]

In [ ]:
def enriched_composition(Hom, steps=1):
    """
    Compute the composed Hom matrix after 'steps' compositions.
    Uses max-product semiring: (max, ×)
    """
    result = Hom.copy()
    for _ in range(steps):
        new_result = np.zeros_like(result)
        for i in range(len(result)):
            for j in range(len(result)):
                # Max over all intermediate objects
                max_path = max(result[i, k] * Hom[k, j] for k in range(len(result)))
                new_result[i, j] = max(result[i, j], max_path)
        result = new_result
    return result

# Compute 2-step composition
Hom_2 = enriched_composition(Hom, steps=1)

print("After one round of composition:")
print(f"  Original non-zero entries: {np.count_nonzero(Hom)}")
print(f"  After composition: {np.count_nonzero(Hom_2 > 0.01)}")

In [ ]:
# Find new reachable pairs after composition
new_pairs = []
for i in range(n):
    for j in range(n):
        if Hom[i, j] < 0.01 and Hom_2[i, j] > 0.01:
            new_pairs.append((all_objects[i], all_objects[j], Hom_2[i, j]))

new_pairs.sort(key=lambda x: -x[2])

print(f"New passages discovered through composition: {len(new_pairs)}")
print("\nTop 10 by probability:")
for src, tgt, prob in new_pairs[:10]:
    print(f"  {src} → {tgt}: {prob:.3f}")

Composition in the enriched category reveals indirect passages that weren't in the original data but can be inferred from chains.

---

## Part 5: The Weighted Graph Perspective

An enriched category is like a weighted directed graph. Let's build one.

In [ ]:
# Build weighted graph
G_weighted = nx.DiGraph()

# Add nodes
G_weighted.add_nodes_from(all_objects)

# Add weighted edges
for _, row in passages.iterrows():
    G_weighted.add_edge(
        row['source_object'], 
        row['target_object'],
        weight=row['weight'],
        frequency=row['frequency']
    )

print(f"Weighted graph: {G_weighted.number_of_nodes()} nodes, {G_weighted.number_of_edges()} edges")

In [ ]:
# Find highest-weight paths
def highest_weight_path(G, source, target):
    """Find the path with highest product of weights."""
    if source == target:
        return [source], 1.0
    
    try:
        # Find all simple paths
        best_path = None
        best_weight = 0
        
        for path in nx.all_simple_paths(G, source, target, cutoff=4):
            # Compute product of edge weights
            weight = 1.0
            for i in range(len(path) - 1):
                edge_data = G.get_edge_data(path[i], path[i+1])
                if edge_data:
                    weight *= edge_data.get('weight', 0)
            
            if weight > best_weight:
                best_weight = weight
                best_path = path
        
        return best_path, best_weight
    except nx.NetworkXNoPath:
        return None, 0.0

# Test on some pairs
test_pairs = [
    ('stakdur_territory', 'reed_marsh'),
    ('deep_dens', 'capital_outskirts'),
    ('egg_chamber', 'breeding_territory')
]

print("Highest-weight paths:")
print("=" * 50)
for src, tgt in test_pairs:
    if src in G_weighted and tgt in G_weighted:
        path, weight = highest_weight_path(G_weighted, src, tgt)
        if path:
            print(f"\n{src} → {tgt}:")
            print(f"  Path: {' → '.join(path)}")
            print(f"  Weight: {weight:.4f}")
        else:
            print(f"\n{src} → {tgt}: No path found")

The weighted graph perspective lets us find the most likely paths through the category.

---

## Part 6: Markov Chains and Stochastic Processes

A probability-enriched category is essentially a Markov chain:
- Objects = states
- Hom(A, B) = transition probability

In [ ]:
# Normalize rows to get a proper transition matrix
row_sums = Hom.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1  # Avoid division by zero
P = Hom / row_sums

print("Transition matrix P (rows sum to 1):")
print(f"  Row sums (should be ~1): min={P.sum(axis=1).min():.2f}, max={P.sum(axis=1).max():.2f}")

In [ ]:
# Simulate a random walk on the category
def random_walk(P, start_idx, steps=10):
    """Simulate a random walk on the Markov chain."""
    path = [start_idx]
    current = start_idx
    
    for _ in range(steps):
        probs = P[current]
        if probs.sum() == 0:
            break
        next_state = np.random.choice(len(probs), p=probs/probs.sum())
        path.append(next_state)
        current = next_state
    
    return path

# Start from stakdur_territory
start_obj = 'stakdur_territory'
if start_obj in obj_to_idx:
    start_idx = obj_to_idx[start_obj]
    
    print(f"Random walks starting from '{start_obj}':")
    print("=" * 50)
    
    for i in range(5):
        path_idx = random_walk(P, start_idx, steps=5)
        path_names = [all_objects[idx] for idx in path_idx]
        print(f"  Walk {i+1}: {' → '.join(path_names)}")

In [ ]:
# Compute stationary distribution
# This is the eigenvector of P^T with eigenvalue 1

try:
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    
    # Find eigenvector with eigenvalue closest to 1
    idx = np.argmin(np.abs(eigenvalues - 1))
    stationary = np.real(eigenvectors[:, idx])
    stationary = stationary / stationary.sum()  # Normalize
    
    # Top stationary probabilities
    top_stationary = sorted(zip(all_objects, stationary), key=lambda x: -x[1])[:10]
    
    print("Stationary distribution (most likely states):")
    print("=" * 50)
    for obj, prob in top_stationary:
        print(f"  {obj}: {prob:.4f}")
except Exception as e:
    print(f"Could not compute stationary distribution: {e}")

The stationary distribution tells us where a random walker spends most time. This reveals the "attractors" in the category.

---

## Part 7: Connection to Attention Mechanisms

In transformers, attention computes weighted relationships between tokens. This is exactly an enriched category:

- Objects = tokens
- Hom(A, B) = attention weight from A to B

Attention is a probability-enriched functor!

In [ ]:
# Simulate attention-like computation
# Keys and queries from object features

# Create simple features: one-hot encoding of morphism types received
morphism_types = passages['morphism_type'].unique()
n_types = len(morphism_types)
type_to_idx = {t: i for i, t in enumerate(morphism_types)}

# Feature matrix: rows = objects, columns = morphism types
features = np.zeros((n, n_types))
for _, row in passages.iterrows():
    j = obj_to_idx[row['target_object']]
    t = type_to_idx[row['morphism_type']]
    features[j, t] += row['weight']

print(f"Object features: {features.shape}")

In [ ]:
# Compute attention-like weights: softmax of dot products
def attention_weights(features):
    """Compute attention matrix from features."""
    # Q, K = features (simplified: use same for both)
    scores = features @ features.T  # (n, n)
    
    # Softmax per row
    exp_scores = np.exp(scores - scores.max(axis=1, keepdims=True))
    attention = exp_scores / exp_scores.sum(axis=1, keepdims=True)
    
    return attention

attention = attention_weights(features)

print("Attention matrix computed")
print(f"  Shape: {attention.shape}")
print(f"  Row sums (should be 1): {attention.sum(axis=1)[:5]}")

In [ ]:
# Compare attention weights to passage probabilities
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

k = 10  # Show first 10 objects

# Original passage weights
ax1 = axes[0]
sns.heatmap(Hom[:k, :k], annot=True, fmt='.2f', cmap='Blues',
            xticklabels=all_objects[:k], yticklabels=all_objects[:k], ax=ax1)
ax1.set_title('Passage Weights (Original)')

# Attention weights
ax2 = axes[1]
sns.heatmap(attention[:k, :k], annot=True, fmt='.2f', cmap='Oranges',
            xticklabels=all_objects[:k], yticklabels=all_objects[:k], ax=ax2)
ax2.set_title('Attention Weights (Learned)')

plt.tight_layout()
plt.show()

Both matrices are enriched Hom structures. The original is from observations; attention is computed from features. Both capture weighted relationships between objects.

---

## Part 8: Enriched Functors

A functor between enriched categories must preserve the enrichment structure.

For probability-enriched categories:
- F: C → D must satisfy: Hom_D(F(A), F(B)) ≥ Hom_C(A, B)
- (The functor doesn't decrease probabilities)

In [ ]:
# Define an enriched functor: aggregation by region
# Objects in same region map to same object

regions = passages['region'].unique()
print(f"Regions: {list(regions)}")

In [ ]:
# Build regional Hom matrix
# Objects = regions
# Hom(R1, R2) = max weight of any passage from R1-object to R2-object

# First, assign objects to regions based on where they're observed
obj_to_region = {}
for _, row in passages.iterrows():
    obj_to_region[row['source_object']] = row['region']
    obj_to_region[row['target_object']] = row['region']

# Build regional Hom
region_list = list(regions)
n_regions = len(region_list)
region_to_idx = {r: i for i, r in enumerate(region_list)}

Hom_regional = np.zeros((n_regions, n_regions))

for _, row in passages.iterrows():
    src_region = row['region']
    tgt_region = obj_to_region.get(row['target_object'], row['region'])
    
    i = region_to_idx[src_region]
    j = region_to_idx.get(tgt_region, i)
    
    Hom_regional[i, j] = max(Hom_regional[i, j], row['weight'])

np.fill_diagonal(Hom_regional, 1.0)

print(f"Regional Hom matrix: {Hom_regional.shape}")

In [ ]:
# Visualize regional Hom
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(Hom_regional, annot=True, fmt='.2f', cmap='Greens',
            xticklabels=region_list, yticklabels=region_list, ax=ax)
ax.set_title('Regional Hom Matrix\n(Enriched Functor from Objects to Regions)')
ax.set_xlabel('Target Region')
ax.set_ylabel('Source Region')
plt.tight_layout()
plt.show()

The regional Hom matrix is the result of an enriched functor that aggregates objects by region. This is how we can "zoom out" while preserving categorical structure.

---

## Part 9: Information Flow and Entropy

The enriched structure tells us about information flow in the category.

In [ ]:
# Compute entropy of outgoing transitions for each object
def entropy(probs):
    """Compute Shannon entropy of a probability distribution."""
    probs = probs[probs > 0]  # Filter zeros
    if len(probs) == 0:
        return 0
    probs = probs / probs.sum()  # Normalize
    return -np.sum(probs * np.log2(probs + 1e-10))

entropies = [entropy(P[i]) for i in range(n)]

# Objects with highest/lowest entropy
entropy_df = pd.DataFrame({'object': all_objects, 'entropy': entropies})
entropy_df = entropy_df.sort_values('entropy', ascending=False)

print("Highest entropy objects (most unpredictable outgoing):")
print(entropy_df.head(10).to_string(index=False))

print("\nLowest entropy objects (most predictable outgoing):")
print(entropy_df.tail(10).to_string(index=False))

In [ ]:
# Visualize entropy distribution
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(range(len(entropies)), sorted(entropies, reverse=True), color='steelblue', alpha=0.7)
ax.set_xlabel('Object (sorted by entropy)')
ax.set_ylabel('Entropy (bits)')
ax.set_title('Entropy of Outgoing Transitions\n(Higher = more unpredictable)')
plt.tight_layout()
plt.show()

High entropy objects have many possible outgoing passages with similar probabilities. Low entropy objects have one dominant passage.

---

## Exercises

### Exercise 1: Cost Enrichment

Instead of probabilities, interpret weights as costs (where higher = more costly). Modify the composition rule to use addition instead of multiplication. Find the lowest-cost paths.

In [ ]:
# Your code here
# Hint: Use min instead of max, and + instead of *

### Exercise 2: PageRank

Implement PageRank on the passage category. This gives a different ranking than the stationary distribution—compare them.

In [ ]:
# Your code here
# Hint: Use nx.pagerank(G_weighted)

### Exercise 3: Morphism Type Weights

Compute separate Hom matrices for each morphism type. How does the categorical structure differ between creature_passage and lifecycle_passage?

In [ ]:
# Your code here
# Hint: Filter passages by morphism_type before building Hom

### Exercise 4: Multi-Head Attention

Implement multi-head attention with 3 heads, where each head focuses on different morphism types. Compare the resulting attention patterns.

In [ ]:
# Your code here
# Hint: Create different feature matrices for different morphism types

---

## Discussion Questions

1. Vold's weighted passages are observational. But attention weights are learned. Does this difference matter for the categorical interpretation?

2. In physics, transition probabilities must be derived from dynamics. In Vold's framework, they're taken as primitive. Is this philosophically defensible?

3. The enriched category framework applies to any "valued" relationships. What other domains (beyond probability and cost) might benefit from this perspective?

---

## Summary

In this tutorial, you learned:

| Concept | What You Learned |
|---------|------------------|
| Enriched category | Hom(A, B) is a value, not just a set |
| Probability enrichment | Morphisms have probabilities; composition multiplies |
| Markov chains | Enriched categories as stochastic processes |
| Attention | Enriched functor computing weighted relationships |
| Stationary distribution | Long-run behavior of random walks |

| Skill | Code Pattern |
|-------|--------------|
| Build Hom matrix | Initialize np.zeros, fill from data |
| Normalize to transition | Divide rows by row sums |
| Compute stationary | Eigenvector of P^T with eigenvalue 1 |
| Attention weights | Softmax of dot products |

---

## Course Conclusion

You have now completed **Relational Foundations**, the first course in the Categorical Philosophy Series.

You learned:
1. **Passage Diagrams**: Categories as objects and morphisms
2. **Classification Hierarchy**: Pre-orders as special categories
3. **Composition and Identity**: The two fundamental operations
4. **The Preservation Principle**: Functors as structure-preserving maps
5. **The Probing Pattern**: Hom functor and representable functors
6. **Coherent Shifts**: Natural transformations between functors
7. **The Probing Theorem**: The Yoneda Lemma and its implications
8. **The Language of Dens**: Language as a category
9. **Weighted Passages**: Enriched categories and probabilistic structure

### Vold's Final Word

> *"We began with a simple question: what are things? I proposed an answer: things are patterns of passages. An object is not defined by what it is, but by how other things pass through it, around it, to it. This is not mysticism—it is mathematics.*
>
> *The Probing Theorem shows that an object is fully determined by its probing pattern. Coherent shifts show how different perspectives relate. Weighted passages capture degrees of possibility. And the language of Dens shows that meaning itself arises from structure.*
>
> *Marden Krell asked: but what are the objects really? I say: there is no 'really.' There are only relationships. And mathematics—category theory—gives us the language to say this precisely."*
> — Tessery Vold, "Reflections on Passage," Year 920

---

## Credits

**Source Material:** Tai-Danae Bradley, "Category Theory and Language Models" (Cartesian Cafe)

**Densworld Integration:** The Relational Foundations course applies categorical concepts through the framework of Tessery Vold.

**Learn more:** [buildLittleWorlds](https://github.com/buildLittleWorlds)

---

> *"A passage is not merely a yes or no. The stakdur's daily hunt is a certainty; the grimslew's rare emergence is a whisper. Some paths are well-worn, others barely visible. To capture the world's structure, we must weight our passages. This is the final refinement: not just what passes, but how much."*
> — Tessery Vold, "Weighted Passages," Year 901